In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr_ca').getOrCreate()

In [0]:
from pyspark.ml.regression import LinearRegression

In [0]:
df = sqlContext.sql("SELECT * FROM ecommerce_customers_1_csv")
df.describe().show()


+-------+-----------------+--------------------+-----------+------------------+------------------+------------------+--------------------+-------------------+
summary| Email| Address| Avatar|Avg Session Length| Time on App| Time on Website|Length of Membership|Yearly Amount Spent|
+-------+-----------------+--------------------+-----------+------------------+------------------+------------------+--------------------+-------------------+
 count| 500| 500| 500| 500| 500| 500| 500| 500|
 mean| null| null| null| 33.05319351819619|12.052487937166134| 37.06044542094859| 3.533461555915055| 499.3140382585909|
 stddev| null| null| null|0.9925631110845354|0.9942156084725424|1.0104889067564033| 0.9992775024112585| 79.3147815497068|
 min|aaron04@yahoo.com|0001 Mack MillNor...| AliceBlue|29.532428967057943| 8.508152176032603| 33.91384724758464| 0.2699010899842742| 256.67058229005585|
 max|zscott@wright.com|Unit 7502 Box 834...|YellowGreen| 36.13966248879052|15.126994288792467|40.005181638101895| 6.922689335035808| 765.5184619388373|
+-------+-----------------+--------------------+-----------+------------------+------------------+------------------+--------------------+-------------------+

In [0]:
df.printSchema()

root
-- Email: string (nullable = true)
-- Address: string (nullable = true)
-- Avatar: string (nullable = true)
-- Avg Session Length: double (nullable = true)
-- Time on App: double (nullable = true)
-- Time on Website: double (nullable = true)
-- Length of Membership: double (nullable = true)
-- Yearly Amount Spent: double (nullable = true)

In [0]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [0]:
df.columns

Out[13]: ['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [0]:
ass = VectorAssembler(inputCols=['Avg Session Length', 'Time on App','Time on Website','Length of Membership'],
                     outputCol='features')

In [0]:
op = ass.transform(df)

In [0]:
op.printSchema()

root
-- Email: string (nullable = true)
-- Address: string (nullable = true)
-- Avatar: string (nullable = true)
-- Avg Session Length: double (nullable = true)
-- Time on App: double (nullable = true)
-- Time on Website: double (nullable = true)
-- Length of Membership: double (nullable = true)
-- Yearly Amount Spent: double (nullable = true)
-- features: vector (nullable = true)

In [0]:
op.head(1)

Out[20]: [Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [0]:
dff = op.select('features','Yearly Amount Spent')
dff.show()

+--------------------+-------------------+
 features|Yearly Amount Spent|
+--------------------+-------------------+
[34.4972677251122...| 587.9510539684005|
[31.9262720263601...| 392.2049334443264|
[33.0009147556426...| 487.54750486747207|
[34.3055566297555...| 581.8523440352177|
[33.3306725236463...| 599.4060920457634|
[33.8710378793419...| 637.102447915074|
[32.0215955013870...| 521.5721747578274|
[32.7391429383803...| 549.9041461052942|
[33.9877728956856...| 570.2004089636196|
[31.9365486184489...| 427.1993848953282|
[33.9925727749537...| 492.6060127179966|
[33.8793608248049...| 522.3374046069357|
[29.5324289670579...| 408.6403510726275|
[33.1903340437226...| 573.4158673313865|
[32.3879758531538...| 470.4527333009554|
[30.7377203726281...| 461.7807421962299|
[32.1253868972878...| 457.84769594494855|
[32.3388993230671...| 407.70454754954415|
[32.1878120459321...| 452.3156754800354|
[32.6178560628234...| 605.061038804892|
+--------------------+-------------------+
only showing top 20 rows

In [0]:
train, test = dff.randomSplit([.7,.3])

In [0]:
train.describe().show()

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 339|
 mean| 498.53769033041306|
 stddev| 74.26720122842092|
 min| 298.76200786180766|
 max| 708.9351848669818|
+-------+-------------------+

In [0]:
test.describe().show()

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 161|
 mean| 500.9487087409026|
 stddev| 89.24031140616682|
 min| 256.67058229005585|
 max| 765.5184619388373|
+-------+-------------------+

In [0]:
lr = LinearRegression(labelCol='Yearly Amount Spent')
lr_model = lr.fit(train)
res=lr_model.evaluate(test)
res.residuals.show()

+-------------------+
 residuals|
+-------------------+
 9.240382534748505|
 -4.138100716246129|
-15.664688043110914|
 -6.500584488641891|
-13.529460169697757|
-21.929577969135266|
 11.953028974054632|
-3.5595613070688614|
 3.026962664307689|
 3.1386954275857306|
 -9.276055068054234|
 -3.896950781031876|
 -6.204285666754117|
 4.7260776519845535|
 18.09865035557823|
 7.139698365959418|
 -4.999798107878917|
 -3.711677890636338|
 -16.50714653701408|
-2.7271184635919212|
+-------------------+
only showing top 20 rows

In [0]:
res.rootMeanSquaredError

Out[31]: 10.035583059030673

In [0]:
res.r2

Out[32]: 0.9872746727811036

In [0]:
dff.describe().show()

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 500|
 mean| 499.3140382585909|
 stddev| 79.3147815497068|
 min| 256.67058229005585|
 max| 765.5184619388373|
+-------+-------------------+

In [0]:
ul = test.select('features')
ul.show()

+--------------------+
 features|
+--------------------+
[29.5324289670579...|
[30.4925366965402...|
[30.8162006488763...|
[31.0613251567161...|
[31.0662181616375...|
[31.1239743499119...|
[31.1695067987115...|
[31.2681042107507...|
[31.3091926408918...|
[31.3662121671876...|
[31.4474464941278...|
[31.5171218025062...|
[31.5257524169682...|
[31.5316044825729...|
[31.6005122003032...|
[31.6548096756927...|
[31.7656188210424...|
[31.8124825597242...|
[31.8164283341993...|
[31.8186165667690...|
+--------------------+
only showing top 20 rows

In [0]:
preds = lr_model.transform(ul)
preds.show()

+--------------------+------------------+
 features| prediction|
+--------------------+------------------+
[29.5324289670579...| 399.399968537879|
[30.4925366965402...|286.60934643616065|
[30.8162006488763...| 281.7510289915799|
[31.0613251567161...| 494.0560425465435|
[31.0662181616375...| 462.4627533773721|
[31.1239743499119...|508.87663180890104|
[31.1695067987115...| 415.4035018282382|
[31.2681042107507...| 427.0300944808928|
[31.3091926408918...|429.69375517562594|
[31.3662121671876...| 427.4501871288992|
[31.4474464941278...|427.87879716327825|
[31.5171218025062...| 279.8153714314176|
[31.5257524169682...| 450.169912476636|
[31.5316044825729...| 431.789528077378|
[31.6005122003032...| 461.0742011355187|
[31.6548096756927...| 468.1237253615891|
[31.7656188210424...|501.55387974348605|
[31.8124825597242...|396.52202287443356|
[31.8164283341993...| 517.6296380406704|
[31.8186165667690...|449.14579183372757|
+--------------------+------------------+
only showing top 20 rows